<a href="https://colab.research.google.com/github/Stephenkkt/Python/blob/main/Python_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Useful Module

In [31]:
#%run LIB.py


import pandas as pd
from functools import reduce
import numpy as np
from datetime import datetime

import folium
#import selenium #pip install SELENIUM

#import streamlit as st #pip install streamlit 
#from streamlit_option_menu import option_menu # 

import plotly.express as px #pip install plotly
from PIL import Image

from geopy.geocoders import Nominatim #pip install geopy
import time
from pprint import pprint

import matplotlib.pyplot as plt
import sqlite3
from sqlite3 import connect

import warnings
warnings.filterwarnings("ignore")


from dateutil import relativedelta #pip install python-dateutil

### 2a. Import Sales file

In [32]:
import pandas as pd

file = 'https://raw.githubusercontent.com/Stephenkkt/Python/main/sales.csv'
sales = pd.read_csv(file, parse_dates = ['date'], header = 0, index_col = 0, dtype = {'Amt':'float'})

sales['mth'] = sales['date'].to_numpy().astype('datetime64[M]')
print("Sales Table")
print(sales.head(5))

print("\nData Type")
print(sales.dtypes)

#Describe the dataset
print("\n")
print(sales.describe(include='all'))

print("\n")
print(sales.nunique(axis=0))

print("\nTotal Sales amount")
print(sum(sales['Amt']))


Sales Table
   Cust_ID  Staff_ID      Amt Prod       date        mth
0      102        45  37500.0    E 2021-01-26 2021-01-01
1       97        10  43500.0    E 2021-01-01 2021-01-01
2      151       225  19500.0    C 2019-10-16 2019-10-01
3      185        54  21000.0    C 2019-08-04 2019-08-01
4       61        88  24500.0    C 2020-02-19 2020-02-01

Data Type
Cust_ID              int64
Staff_ID             int64
Amt                float64
Prod                object
date        datetime64[ns]
mth         datetime64[ns]
dtype: object


             Cust_ID      Staff_ID           Amt   Prod                 date  \
count   10000.000000  10000.000000  10000.000000  10000                10000   
unique           NaN           NaN           NaN     10                 1000   
top              NaN           NaN           NaN      B  2019-11-05 00:00:00   
freq             NaN           NaN           NaN   1041                   22   
first            NaN           NaN           NaN    NaN  

## 2b. Plot Simple Bar Chart - By Product

In [33]:
pd.options.plotting.backend = "plotly"
sales.groupby(by=["Prod"]).sum()[["Amt"]].plot(kind="bar")

## 3. Import Product File

In [34]:
prod = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/Prod.csv', header = 0, index_col=0)

print("Product:\n")
print(prod.head(5))

print("\nData Type")
print(prod.dtypes)

Product:

             Cat
Prod            
A     Category 1
B     Category 1
C     Category 1
D     Category 1
E     Category 2

Data Type
Cat    object
dtype: object


## 4a. Import Branch File

In [35]:
branch = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/BR_F.csv', header = 0, index_col = 0)

print("BRANCH:\n")
print(branch.head(5))
print("\nData Type")
print(branch.dtypes)



BRANCH:

                        Name                 Address     Zip         Phone  \
index                                                                        
1             1200 St Clair   1200 ST CLAIR AVE WEST  M6E1B4  416-652-3444   
3      Avenue Rd & Fairlawn         1705 AVENUE ROAD  M5M3Y5  416-782-3508   
5      Bathurst & Frontenac         2953 BATHURST ST  M6B3B2  416-789-7915   
10            Bay & College           763 BAY STREET  M5G2R3  416-867-5190   
13          Bedford & Bloor    242 BLOOR STREET WEST  M5S1T8  416-867-4663   

       Br_ID  
index         
1          1  
3          2  
5          3  
10         4  
13         5  

Data Type
Name       object
Address    object
Zip        object
Phone      object
Br_ID       int64
dtype: object


## 4b. Clean branch address

In [36]:
#Clean Postal Code
branch['postal'] = branch['Zip'].str[:3] + " " + branch['Zip'].str[-3:]
branch = branch.drop(columns = ['Zip'])

#Create new field 'full address'
branch['full_address'] = branch['Address'] + "," + " Ontario" + "," + " Canada"

print('\nBranch')
print(branch.head())


Branch
                        Name                 Address         Phone  Br_ID  \
index                                                                       
1             1200 St Clair   1200 ST CLAIR AVE WEST  416-652-3444      1   
3      Avenue Rd & Fairlawn         1705 AVENUE ROAD  416-782-3508      2   
5      Bathurst & Frontenac         2953 BATHURST ST  416-789-7915      3   
10            Bay & College           763 BAY STREET  416-867-5190      4   
13          Bedford & Bloor    242 BLOOR STREET WEST  416-867-4663      5   

        postal                             full_address  
index                                                    
1      M6E 1B4  1200 ST CLAIR AVE WEST, Ontario, Canada  
3      M5M 3Y5        1705 AVENUE ROAD, Ontario, Canada  
5      M6B 3B2        2953 BATHURST ST, Ontario, Canada  
10     M5G 2R3          763 BAY STREET, Ontario, Canada  
13     M5S 1T8   242 BLOOR STREET WEST, Ontario, Canada  



## 4c. Map branch coordination

In [37]:
from geopy.geocoders import Nominatim #pip install geopy

#find the coordination
geolocator = Nominatim(timeout=10, user_agent = "MyApp")
branch['gcode'] = branch.full_address.apply(geolocator.geocode)

branch['lat'] = [g.latitude for g in branch.gcode]
branch['long'] = [g.longitude for g in branch.gcode]
print("\nBranch with Coordinate")
print(branch.head())



Branch with Coordinate
                        Name                 Address         Phone  Br_ID  \
index                                                                       
1             1200 St Clair   1200 ST CLAIR AVE WEST  416-652-3444      1   
3      Avenue Rd & Fairlawn         1705 AVENUE ROAD  416-782-3508      2   
5      Bathurst & Frontenac         2953 BATHURST ST  416-789-7915      3   
10            Bay & College           763 BAY STREET  416-867-5190      4   
13          Bedford & Bloor    242 BLOOR STREET WEST  416-867-4663      5   

        postal                             full_address  \
index                                                     
1      M6E 1B4  1200 ST CLAIR AVE WEST, Ontario, Canada   
3      M5M 3Y5        1705 AVENUE ROAD, Ontario, Canada   
5      M6B 3B2        2953 BATHURST ST, Ontario, Canada   
10     M5G 2R3          763 BAY STREET, Ontario, Canada   
13     M5S 1T8   242 BLOOR STREET WEST, Ontario, Canada   

                      

## 5. Import Staff File

In [38]:
staff = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/staff.csv', header = 0, index_col = 0)

#staff = staff.reset_index()
print("STAFF:\n")
print(staff.head(5))

print("\nData Type")
print(staff.dtypes)

print("\n")
print(staff.nunique(axis=0))

STAFF:

          Br_ID
Staff_ID       
1            27
2            20
3            42
4            12
5            28

Data Type
Br_ID    int64
dtype: object


Br_ID    51
dtype: int64


## Import Target File

In [39]:
Tar = pd.read_csv('https://raw.githubusercontent.com/Stephenkkt/Python/main/BR_TAR.csv', header = 0, index_col=0, dtype = {'Br_ID':'int64'})
Tar['Sales_Tar'] = Tar['Sales_Tar'].replace(',','',regex=True)
Tar['Sales_Tar'] = Tar['Sales_Tar'].astype('float')
Tar = Tar.reset_index(drop=False)
print(Tar.head())


print("\nTotal Sales Target")
print(sum(Tar['Sales_Tar']))

   Br_ID  Sales_Tar
0      1  3791040.0
1      2  4963200.0
2      3  7075200.0
3      4  3199680.0
4      5  3759360.0

Total Sales Target
268213440.0


## 6. Summary

* **a.Summary in Staff Level**

In [40]:
#Summary in Staff Level
sum_sales_staff_1 = sales.groupby(['Staff_ID'], dropna = False).agg(Total=('Amt','sum'),
                                                    Count=('Amt','count'),
                                                    Avg_Tix=('Amt','mean')
                                                   )
print("\nSales by Staff")
print(sum_sales_staff_1)

print("\nCheck total sales")
print(sum(sum_sales_staff_1['Total']))



Sales by Staff
              Total  Count       Avg_Tix
Staff_ID                                
1          895000.0     36  24861.111111
2         1091500.0     41  26621.951220
3          705000.0     30  23500.000000
4          694500.0     32  21703.125000
5          895000.0     31  28870.967742
...             ...    ...           ...
296        866000.0     32  27062.500000
297        873500.0     34  25691.176471
298        561500.0     24  23395.833333
299       1297500.0     47  27606.382979
300        797500.0     30  26583.333333

[300 rows x 3 columns]

Check total sales
252539500.0


* **Merge Staff Table for getting Branch ID**

In [41]:
sum_sales_br_1 = pd.merge(sum_sales_staff_1, staff, how = 'inner', left_on = 'Staff_ID', right_on = 'Staff_ID')

print("\nSales by Staff with Branch")
print(sum_sales_br_1)

print("\nCheck total sales")
print(sum(sum_sales_br_1['Total']))



Sales by Staff with Branch
              Total  Count       Avg_Tix  Br_ID
Staff_ID                                       
1          895000.0     36  24861.111111     27
2         1091500.0     41  26621.951220     20
3          705000.0     30  23500.000000     42
4          694500.0     32  21703.125000     12
5          895000.0     31  28870.967742     28
...             ...    ...           ...    ...
296        866000.0     32  27062.500000     39
297        873500.0     34  25691.176471      6
298        561500.0     24  23395.833333     13
299       1297500.0     47  27606.382979     41
300        797500.0     30  26583.333333     38

[300 rows x 4 columns]

Check total sales
252539500.0


* **Summary in Branch Level**

In [42]:
sum_sales_br_2 = sum_sales_br_1.groupby(['Br_ID'], dropna = False).agg(BR_Total = ('Total', 'sum'),
                                                    BR_Count = ('Count','sum')
                                                    )
sum_sales_br_2['BR_Avg_Tix']=sum_sales_br_2['BR_Total']/sum_sales_br_2['BR_Count']
print("\nSales by Branch")
print(sum_sales_br_2.head())
print("\nSales by Branch - Number of Record")
print(len(sum_sales_br_2))
print("\nCheck total sales")
print(sum(sum_sales_br_2['BR_Total']))



Sales by Branch
        BR_Total  BR_Count    BR_Avg_Tix
Br_ID                                   
1      3453500.0       136  25393.382353
2      4690000.0       189  24814.814815
3      6390000.0       264  24204.545455
4      2446500.0       104  23524.038462
5      3600000.0       133  27067.669173

Sales by Branch - Number of Record
51

Check total sales
252539500.0


* **Merge Branch Table for Brnach details and create rank**

In [43]:
sum_sales_br_2 = pd.merge(sum_sales_br_2,branch, how = 'inner', left_on = 'Br_ID', right_on = 'Br_ID')

#Create Ranking
sum_sales_br_2['rank'] = sum_sales_br_2['BR_Total'].rank(ascending = False).astype(int)

print("\nSales by Branch")
print(sum_sales_br_2.head())

print("\nSales by Branch - Number of Record")
print(len(sum_sales_br_2))

print("\nCheck total sales")
print(sum(sum_sales_br_2['BR_Total']))



Sales by Branch
   Br_ID   BR_Total  BR_Count    BR_Avg_Tix                   Name  \
0      1  3453500.0       136  25393.382353         1200 St Clair    
1      2  4690000.0       189  24814.814815  Avenue Rd & Fairlawn    
2      3  6390000.0       264  24204.545455  Bathurst & Frontenac    
3      4  2446500.0       104  23524.038462         Bay & College    
4      5  3600000.0       133  27067.669173       Bedford & Bloor    

                  Address         Phone   postal  \
0  1200 ST CLAIR AVE WEST  416-652-3444  M6E 1B4   
1        1705 AVENUE ROAD  416-782-3508  M5M 3Y5   
2        2953 BATHURST ST  416-789-7915  M6B 3B2   
3          763 BAY STREET  416-867-5190  M5G 2R3   
4   242 BLOOR STREET WEST  416-867-4663  M5S 1T8   

                              full_address  \
0  1200 ST CLAIR AVE WEST, Ontario, Canada   
1        1705 AVENUE ROAD, Ontario, Canada   
2        2953 BATHURST ST, Ontario, Canada   
3          763 BAY STREET, Ontario, Canada   
4   242 BLOOR STREE

## Summary by date

In [44]:
sum_sales_date = sales.groupby('date', dropna = False).agg(Total = ('Amt', 'sum'),
                                           Count = ('Amt', 'count')
                                          )
sum_sales_date = sum_sales_date.reset_index(drop = False)
print("Sale by Date")
print(sum_sales_date)

print("\nCheck total sales")
print(sum(sum_sales_date['Total']))


Sale by Date
          date     Total  Count
0   2019-01-02  294000.0     10
1   2019-01-03  200500.0      7
2   2019-01-04  115000.0      7
3   2019-01-05  348000.0     12
4   2019-01-06  395500.0     12
..         ...       ...    ...
995 2021-09-23  162000.0      9
996 2021-09-24  156000.0      6
997 2021-09-25  267500.0     10
998 2021-09-26  318500.0     12
999 2021-09-27   80000.0      4

[1000 rows x 3 columns]

Check total sales
252539500.0


## Summary by Category, Product and month

* **Summary by Product and month**

In [45]:
sum_sales_prod_mth = sales.groupby(['mth','Prod'], dropna = False).agg(Total = ("Amt","sum"),
                                                       Count = ("Amt", "count")
                                                      )
sum_sales_prod_mth = sum_sales_prod_mth.reset_index(drop = False)
print("\nSale by Prod and mth")
print(sum_sales_prod_mth)

print("\nCheck total sales")
print(sum(sum_sales_prod_mth['Total']))



Sale by Prod and mth
           mth Prod      Total  Count
0   2019-01-01    A  1014000.0     40
1   2019-01-01    B   460500.0     19
2   2019-01-01    C   679500.0     26
3   2019-01-01    D   636500.0     24
4   2019-01-01    E   688500.0     21
..         ...  ...        ...    ...
325 2021-09-01    F   490000.0     18
326 2021-09-01    G   534500.0     27
327 2021-09-01    H   594000.0     24
328 2021-09-01    I   838000.0     31
329 2021-09-01    J   896500.0     35

[330 rows x 4 columns]

Check total sales
252539500.0


* **Merge Product Table**

In [46]:
sum_sales_prod_mth = pd.merge(sum_sales_prod_mth,prod, how = 'inner', left_on = "Prod", right_on = "Prod")

print("\nSale by Prod and mth")
print(sum_sales_prod_mth)


Sale by Prod and mth
           mth Prod      Total  Count         Cat
0   2019-01-01    A  1014000.0     40  Category 1
1   2019-02-01    A   566500.0     23  Category 1
2   2019-03-01    A   782500.0     38  Category 1
3   2019-04-01    A   665500.0     27  Category 1
4   2019-05-01    A   867000.0     38  Category 1
..         ...  ...        ...    ...         ...
325 2021-05-01    J   536500.0     24  Category 3
326 2021-06-01    J  1039500.0     34  Category 3
327 2021-07-01    J   727500.0     28  Category 3
328 2021-08-01    J   987500.0     40  Category 3
329 2021-09-01    J   896500.0     35  Category 3

[330 rows x 5 columns]


## Branch sales in latest month

In [47]:
curr_mth_sales = sales.copy()

print("\nCheck total sales - 1")
print(sum(sales['Amt']))

curr_mth_sales2 = curr_mth_sales.loc[(sales['date'].max().to_numpy().astype('datetime64[M]') == sales['date'].to_numpy().astype('datetime64[M]'))]

print("\nCheck total sales - 2")
print(sum(curr_mth_sales2['Amt']))

print("\nmin=" + str(curr_mth_sales2['date'].min()))
print("\nmax="+str(curr_mth_sales2['date'].max()))

sum_curr_mth_1 = curr_mth_sales2.groupby(['Staff_ID'], dropna = False).agg(Total = ('Amt', 'sum'),
                                                           Count = ('Amt', 'count')
                                                            )

print("\nCheck total sales - 3")
print(sum(sum_curr_mth_1['Total']))

sum_curr_mth_1 = pd.merge(sum_curr_mth_1, staff, how = 'inner', left_on = 'Staff_ID', right_on = 'Staff_ID')


sum_curr_mth_2 = sum_curr_mth_1.groupby(['Br_ID'], dropna = False).agg(Total = ('Total', 'sum'),
                                                           Count = ('Count', 'sum')
                                                            )

print("\nCheck total sales - 4")
print(sum(sum_curr_mth_2['Total']))

sum_curr_mth_2 = pd.merge(sum_curr_mth_2,Tar, how = 'inner', left_on='Br_ID', right_on='Br_ID')
sum_curr_mth_2 = pd.merge(sum_curr_mth_2,branch[['Br_ID','Name']], how = 'inner', left_on='Br_ID', right_on='Br_ID')

print(sum_curr_mth_2.head())

print("\nCheck total sales - 5")
print(sum(sum_curr_mth_2['Total']))


Check total sales - 1
252539500.0

Check total sales - 2
6481500.0

min=2021-09-01 00:00:00

max=2021-09-27 00:00:00

Check total sales - 3
6481500.0

Check total sales - 4
6481500.0
   Br_ID     Total  Count  Sales_Tar                   Name
0      1   60500.0      3  3791040.0         1200 St Clair 
1      2   72000.0      4  4963200.0  Avenue Rd & Fairlawn 
2      3  124500.0      6  7075200.0  Bathurst & Frontenac 
3      4   95000.0      5  3199680.0         Bay & College 
4      5  160000.0      5  3759360.0       Bedford & Bloor 

Check total sales - 5
6481500.0


## Sales by Branch, Category and Product (Current month only)

### Use "curr_mth_sales2" generated

#### Merge Product Table, Staff Table and Branch Name

In [48]:
curr_mth_sales3 = pd.merge(curr_mth_sales2, prod, how = "inner", left_on="Prod", right_on="Prod")
curr_mth_sales3 = pd.merge(curr_mth_sales3, staff, how = "inner", left_on="Staff_ID", right_on="Staff_ID")
curr_mth_sales3 = pd.merge(curr_mth_sales3, branch[["Br_ID","Name"]], how = "inner", left_on="Br_ID", right_on="Br_ID")

sum_curr_detail = curr_mth_sales3.groupby(['Br_ID','Name','Cat','Prod'], dropna = False).agg(Total= ('Amt','sum'),
                                                                   Count= ('Amt', 'count') 
                                                                  )
sum_curr_detail=sum_curr_detail.reset_index()
print(sum_curr_detail.head())

print("\nCheck total sales")
print(sum(sum_curr_detail['Total']))

   Br_ID                   Name         Cat Prod    Total  Count
0      1         1200 St Clair   Category 2    G  35000.0      2
1      1         1200 St Clair   Category 3    J  25500.0      1
2      2  Avenue Rd & Fairlawn   Category 1    A  64000.0      3
3      2  Avenue Rd & Fairlawn   Category 1    C   8000.0      1
4      3  Bathurst & Frontenac   Category 1    A  32500.0      1

Check total sales
6481500.0


In [49]:
## Raw Summary File For Tableau

raw_sum = sales.groupby(['date','Staff_ID','Prod']).agg(Total=('Amt', 'sum'),
                                                      Count= ('Amt', 'count'))
raw_sum = raw_sum.reset_index()

print(raw_sum.head())

raw_sum = pd.merge(raw_sum, prod, how = "inner", left_on="Prod", right_on="Prod")
raw_sum = pd.merge(raw_sum, staff, how = "inner", left_on="Staff_ID", right_on="Staff_ID")

raw_sum = pd.merge(raw_sum, branch[["Br_ID","Name","lat","long"]], how = "inner", left_on="Br_ID", right_on="Br_ID")
print(raw_sum.head())

print("\nCheck total sales")
print(sum(raw_sum['Total']))


        date  Staff_ID Prod    Total  Count
0 2019-01-02         6    C   1000.0      1
1 2019-01-02        65    G  46000.0      1
2 2019-01-02        82    F  30500.0      1
3 2019-01-02       101    A  45500.0      1
4 2019-01-02       123    G  12000.0      1
        date  Staff_ID Prod    Total  Count         Cat  Br_ID  \
0 2019-01-02         6    C   1000.0      1  Category 1      7   
1 2019-06-07         6    C  20500.0      1  Category 1      7   
2 2019-07-11         6    C  41000.0      1  Category 1      7   
3 2020-04-04         6    C   1000.0      1  Category 1      7   
4 2020-11-13         6    C  34500.0      1  Category 1      7   

              Name        lat       long  
0  Bloor & Church   43.670599 -79.385153  
1  Bloor & Church   43.670599 -79.385153  
2  Bloor & Church   43.670599 -79.385153  
3  Bloor & Church   43.670599 -79.385153  
4  Bloor & Church   43.670599 -79.385153  

Check total sales
252539500.0


## Export Files

In [50]:
sum_sales_prod_mth.to_csv('sum_prod_mth.csv', index = False, header = 1)
sum_sales_br_2.to_csv('sum_branch.csv', index = False, header = 1)
sum_sales_date.to_csv('sum_date.csv', index = False, header = 1)
sum_curr_mth_2.to_csv('sum_achievement.csv', index = False, header = 1)
sum_curr_detail.to_csv('sum_curr_mth_detail.csv', index = False, header = 1)
raw_sum.to_csv('sum_tableau.csv', index = False, header = 1)

## Show all generated dataframe

In [51]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
Tar                  DataFrame        Br_ID   Sales_Tar\n0 <...>.0\n50     51   4097280.0
branch               DataFrame                             <...>.  43.774283 -79.501947  
curr_mth_sales       DataFrame          Cust_ID  Staff_ID  <...>n[10000 rows x 6 columns]
curr_mth_sales2      DataFrame          Cust_ID  Staff_ID  <...>n\n[253 rows x 6 columns]
curr_mth_sales3      DataFrame         Cust_ID  Staff_ID   <...>n\n[253 rows x 9 columns]
prod                 DataFrame                 Cat\nProd   <...>egory 3\nJ     Category 3
raw_sum              DataFrame               date  Staff_ID<...>n[9981 rows x 10 columns]
sales                DataFrame          Cust_ID  Staff_ID  <...>n[10000 rows x 6 columns]
staff                DataFrame              Br_ID\nStaff_ID<...>n\n[300 rows x 1 columns]
sum_curr_detail      DataFrame         Br_ID               <...>n\n[198 rows x 6 columns]
sum_curr_mth

## 7. Create Map

In [52]:
import folium

# Find the Latitude & Longtitude for the map
avg_lat=branch['lat'].mean()
avg_long=branch['long'].mean()

print(avg_lat)
print(avg_long)

map = folium.Map(
    height=700, width=700,
    titles='Branch Sales',
    location=[avg_lat, avg_long],
    zoom_start = 11
)
map


43.69675561200355
-79.36020879238686


In [53]:
# Map Colour 
def sales_rank(row):
    if row['rank'] <=5:
        return 'pink'
    elif row['rank'] >=46:
        return 'red'
    else:
        return 'blue'

def sales_icon(row):
    if row['rank'] <=5:
        return 'star'
    elif row['rank'] >=46:
        return 'exclamation'
    else:
        return 'fa-circle'

sum_sales_br_2['colors'] = sum_sales_br_2.apply(sales_rank, axis=1)
sum_sales_br_2['icons'] = sum_sales_br_2.apply(sales_icon, axis=1)
sum_sales_br_2['BR_Total_M']= "$" + (round(sum_sales_br_2['BR_Total']/100000,1)).astype(str) + "M"
print(sum_sales_br_2['icons'].unique())
print(sum_sales_br_2['colors'].unique())

['fa-circle' 'exclamation' 'star']
['blue' 'red' 'pink']


In [54]:
city = sum_sales_br_2.iloc[0]

folium.Marker(
    location=[city['lat'], city['long']],
).add_to(map)


for _, city in sum_sales_br_2.iterrows():
    folium.Marker(
        location = [city['lat'], city['long']],
        popup = [city['Name']],
        tooltip = [city['Name'],
                                      
                   "Sales: ",city['BR_Total_M'],
                   
                   'Rank: ',city['rank']],
        icon=folium.Icon(color=city['colors'], icon=city['icons'], prefix='fa')
    ).add_to(map)
map


### Export Map as html

In [55]:
#map.save('c:/users/user/python_trial/Branch_Sales.html')